In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [120]:
import os
import numpy as np
import requests, zipfile, io
from IPython.display import Image
import ee


In [121]:
data = pd.read_csv(
    "data/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_Nep_AQI_Dataset.csv"
)
data.shape


(12240, 14)

In [122]:
def normalize_filename(filename):
    splits = filename.split(".")
    return splits[0] + "." + splits[1][:2]


data["Normalized_Filename"] = data["Filename"].apply(normalize_filename)


In [123]:
coordinates = [
    ("ITO, Delhi", 28.6284, 77.2425),
    ("Dimapur, Nagaland", 25.9094, 93.7276),
    ("Spice Garden, Bengaluru", 12.9724, 77.5807),
    ("Knowledge Park, Greater Noida", 28.4674, 77.503),
    ("New Ind Town, Faridabad", 28.4089, 77.3178),
    ("Borivali East, Mumbai", 19.2300, 72.8602),
    ("Oragadam, Tamil Nadu", 12.8321, 79.9943),
    ("Biratnagar, Nepal", 26.4525, 87.2718) 
]


In [124]:
def map_locations(data, coordinates):
    lat = [0] * len(data["Location"])
    lon = [0] * len(data["Location"])
    for i, location in enumerate(data["Location"]):
        for j, coord in enumerate(coordinates):
            if location.lower() in coord[0].lower():
                lat[i] = coord[1]
                lon[i] = coord[2]

    return lat, lon


lat, lon = map_locations(data, coordinates)

data["Latitude"] = lat
data["Longitude"] = lon

data['Latitude'].value_counts()


Latitude
19.2300    2203
26.4525    2171
28.6284    2168
12.9724    1926
12.8321    1540
28.4674    1241
28.4089     641
25.9094     350
Name: count, dtype: int64

In [125]:

data[data["Latitude"] == 0]


,Location,Filename,Year,Month,Day,Hour,AQI,PM2.5,PM10,O3,CO,SO2,NO2,AQI_Class,Normalized_Filename,Latitude,Longitude


In [126]:
unique_data = data[~data["Normalized_Filename"].str.contains(".jp")]
unique_data.head()

,Location,Filename,Year,Month,Day,Hour,AQI,PM2.5,PM10,O3,CO,SO2,NO2,AQI_Class,Normalized_Filename,Latitude,Longitude
0,"Biratnagar, Nepal",BRI_Un_2023-02-02- 12.00-9.jpg,2023,2,2,12:00,158,70.08,100.82,58.89,0.49,4.4,1.04,d_Unhealthy,BRI_Un_2023-02-02- 12.00,26.4525,87.2718
1,"Biratnagar, Nepal",BRI_Un_2023-02-02- 12.00-8.jpg,2023,2,2,12:00,158,70.08,100.82,58.89,0.49,4.4,1.04,d_Unhealthy,BRI_Un_2023-02-02- 12.00,26.4525,87.2718
2,"Biratnagar, Nepal",BRI_Un_2023-02-02- 12.00-7.jpg,2023,2,2,12:00,158,70.08,100.82,58.89,0.49,4.4,1.04,d_Unhealthy,BRI_Un_2023-02-02- 12.00,26.4525,87.2718
3,"Biratnagar, Nepal",BRI_Un_2023-02-02- 12.00-6.jpg,2023,2,2,12:00,158,70.08,100.82,58.89,0.49,4.4,1.04,d_Unhealthy,BRI_Un_2023-02-02- 12.00,26.4525,87.2718
4,"Biratnagar, Nepal",BRI_Un_2023-02-02- 12.00-5.jpg,2023,2,2,12:00,158,70.08,100.82,58.89,0.49,4.4,1.04,d_Unhealthy,BRI_Un_2023-02-02- 12.00,26.4525,87.2718


In [127]:
non_unique_data = data[data["Normalized_Filename"].str.contains(".jp")]
non_unique_data.head()

,Location,Filename,Year,Month,Day,Hour,AQI,PM2.5,PM10,O3,CO,SO2,NO2,AQI_Class,Normalized_Filename,Latitude,Longitude
573,"Biratnagar, Nepal",BRI_Un_2023-02-02-1700-9.jpg,2023,2,2,17:00,156,67.06,95.93,52.38,0.52,3.35,6.49,d_Unhealthy,BRI_Un_2023-02-02-1700-9.jp,26.4525,87.2718
574,"Biratnagar, Nepal",BRI_Un_2023-02-02-1700-8.jpg,2023,2,2,17:00,156,67.06,95.93,52.38,0.52,3.35,6.49,d_Unhealthy,BRI_Un_2023-02-02-1700-8.jp,26.4525,87.2718
575,"Biratnagar, Nepal",BRI_Un_2023-02-02-1700-7.jpg,2023,2,2,17:00,156,67.06,95.93,52.38,0.52,3.35,6.49,d_Unhealthy,BRI_Un_2023-02-02-1700-7.jp,26.4525,87.2718
576,"Biratnagar, Nepal",BRI_Un_2023-02-02-1700-6.jpg,2023,2,2,17:00,156,67.06,95.93,52.38,0.52,3.35,6.49,d_Unhealthy,BRI_Un_2023-02-02-1700-6.jp,26.4525,87.2718
577,"Biratnagar, Nepal",BRI_Un_2023-02-02-1700-5.jpg,2023,2,2,17:00,156,67.06,95.93,52.38,0.52,3.35,6.49,d_Unhealthy,BRI_Un_2023-02-02-1700-5.jp,26.4525,87.2718


In [128]:
def normalize_filename_non_unique(filename: str):
    idx = filename.rfind("-")
    return filename[:idx]

In [129]:
non_unique_data["Normalized_Filename"] = non_unique_data["Normalized_Filename"].apply( normalize_filename_non_unique)

C:\Users\Yuval\AppData\Local\Temp\ipykernel_17388\234130207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_unique_data["Normalized_Filename"] = non_unique_data["Normalized_Filename"].apply( normalize_filename_non_unique)


In [130]:
# non_unique_data.head()

In [131]:
# new_data = pd.concat([unique_data, non_unique_data], axis=0)
# I want the data to be in the same order as the original data
new_data = pd.concat([unique_data, non_unique_data], axis=0).sort_index()

In [132]:
new_data['Normalized_Filename'].unique()

array(['BRI_Un_2023-02-02- 12.00', 'BIR_UNH_VF_2023-02-02- 12.00',
       'BRI_Un_2023-02-02- 15.00', 'BIR_UNH_VF_2023-02-02- 15.00',
       'BRI_Un_2023-02-02-1700', 'BIR_UNH_VF_2023-02-02- 1700-3',
       'BIR_UNH_VF_2023-02-02- 1700-2', 'BRI_Un_2023-02-03- 10.00',
       'BIR_UNH_VF_2023-02-03- 10.00', 'BRI_UFSG_2023-02-03- 15.00',
       'BIR_UNFSG_VF_2023-02-03-15.00', 'BRI_UFSG_2023-02-03- 16.00',
       'BIR_UNFSG_VF_2023-02-03-16.00', 'BRI_Un_2023-02-09-08.00',
       'BIR_MOD_2023-02-13-09.30', 'BIR_UFSG_2023-02-14- 14.30',
       'BIR_UH_2023-03-16-12.30', 'TN_UHFSG_2023-02-02- 09.00',
       'TN_UNH_2023-02-03-12.30', 'TN_UNH_2023-02-03-13.30',
       'TN_MOD_2023-02-09-08.30', 'TN_UHFSG_2023-02-10-08.30',
       'TN_Good_2023-02-20-08.30', 'TN_Good_2023-02-20-16.30',
       'TN_Good_2023-02-21-08.30', 'TN_Good_2023-02-22-09.30',
       'TN_Good_2023-02-23-08.30', 'TN_Good_2023-02-27-16.30',
       'TN_Mod_2023-02-28-08.30', 'TN_UnFSG_2023-03-01-08.30',
       'TN_UnFSG_2023

In [133]:
# new_data.iloc[non_unique_data.index]

In [134]:
ee.Authenticate()  # Requires Earth Engine account
ee.Initialize(project="ee-yuvalmehta728")

image_res = 30  # Resolution in meters
n_pixels = 224  # Number of pixels
# image_res = 10  # Higher resolution (15 meters per pixel)
# n_pixels = 600  # Smaller area (112x112 pixels


In [135]:
def mask_l8_sr(image):
    """Applies cloud masking to Landsat 8 Collection 2 data."""
    cloud_shadow_bitmask = 1 << 4
    clouds_bitmask = 1 << 3

    # Extract QA_PIXEL band
    qa_pixel = image.select("QA_PIXEL")

    # Create a mask based on the bitmask values for clouds and cloud shadows
    mask = (
        qa_pixel.bitwiseAnd(cloud_shadow_bitmask)
        .eq(0)
        .And(qa_pixel.bitwiseAnd(clouds_bitmask).eq(0))
    )

    # Apply mask and scale optical bands to reflectance
    return (
        image.updateMask(mask)
        .select(["SR_B2", "SR_B3", "SR_B4"])
        .multiply(0.0000275)
        .add(-0.2)
    )


def get_rectangle(point, image_res, n_pixels):
    """Generates a bounding box of image_res * n_pixels meters around a point."""
    length = image_res * n_pixels
    region = point.buffer(length / 2).bounds().getInfo()["coordinates"]
    coords = np.array(region)
    coords = [
        np.min(coords[:, :, 0]),
        np.min(coords[:, :, 1]),
        np.max(coords[:, :, 0]),
        np.max(coords[:, :, 1]),
    ]
    return ee.Geometry.Rectangle(coords)


def visualization(point, name, landsat, vis_params=None):
    """Visualizes Landsat images with or without cloud masking."""
    if vis_params is None:
        vis_params = {
            "min": 0,
            "max": 0.3,
            "gamma": 1.4,
            "bands": ["SR_B4", "SR_B3", "SR_B2"],
            #   'bands': ['B4', 'B3', 'B2'],
            "dimensions": f"{n_pixels}x{n_pixels}",
            "format": "jpg",
        }

    # Generate bounding box
    rectangle = get_rectangle(point, image_res, n_pixels)

    # Select the appropriate dataset
    image = landsat.mean().clip(rectangle)

    # Get the thumbnail URL and download the image
    try:
       thumb_url = image.getThumbUrl(vis_params)
       response = requests.get(thumb_url)
       response.raise_for_status()
       with open(f"{name}.jpg", "wb") as f:
           f.write(response.content)

       return [True, f"{name}.jpg"]
    except requests.exceptions.RequestException as e:
       print(f"Error fetching image: {e}")
       return [False, None]


In [136]:
# startDate = '2020-01-01'
# endDate = '2020-12-31'

# landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
#     .filterDate(startDate, endDate)
# landsat_masked = landsat.map(mask_l8_sr)
# mask = True

# oval = ee.Geometry.Point([77.5946, 12.9716]) 
# name = 'tryial'
# print(visualization(point=oval, name=name, landsat=(landsat if not mask else landsat_masked), vis_params=None))
# Image(filename=f'{name}.jpg') 


In [137]:
def check_image_exists(filename):
    return os.path.exists(f"images/{filename}")

def get_date_time(data_point):
    hour, minute = data_point["Hour"].split(":")
    return datetime(year=int(data_point["Year"]), month=int(data_point["Month"]), day=int(data_point["Day"]), hour=int(hour), minute=int(minute))

In [138]:
mask = True
base_path = "images3"
visited = []
errors = []
for i in range(0, len(new_data)):
    image_filename = new_data["Normalized_Filename"].iloc[i]
    if check_image_exists(image_filename) or image_filename in visited:
        
        continue
    try:
        print(new_data['Location'].iloc[i])
        
        date = get_date_time(new_data.iloc[i])
        startDate = date - pd.DateOffset(years=1)
        startDate = startDate.strftime("%Y-%m-%d")
        # endDate = date + pd.DateOffset(
        endDate = date.strftime("%Y-%m-%d")
    
        landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterDate(startDate, endDate)
        landsat_masked = landsat.map(mask_l8_sr)
        
        point = ee.Geometry.Point([new_data["Longitude"].iloc[i], new_data["Latitude"].iloc[i]])
        
        name = os.path.join(base_path,image_filename)
        print(visualization(point=point, name=name, landsat=(landsat if not mask else landsat_masked), vis_params=None))
        visited.append(image_filename)
        # Image(filename=f'{name}.jpg') 
    except Exception as e:
        print(e)
        visited.append(image_filename)
        errors.append(i)
        continue
    # break

Biratnagar, Nepal
[True, 'images3\\BRI_Un_2023-02-02- 12.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_UNH_VF_2023-02-02- 12.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BRI_Un_2023-02-02- 15.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_UNH_VF_2023-02-02- 15.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BRI_Un_2023-02-02-1700.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_UNH_VF_2023-02-02- 1700-3.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_UNH_VF_2023-02-02- 1700-2.jpg']
Biratnagar, Nepal
[True, 'images3\\BRI_Un_2023-02-03- 10.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_UNH_VF_2023-02-03- 10.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BRI_UFSG_2023-02-03- 15.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_UNFSG_VF_2023-02-03-15.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BRI_UFSG_2023-02-03- 16.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_UNFSG_VF_2023-02-03-16.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BRI_Un_2023-02-09-08.00.jpg']
Biratnagar, Nepal
[True, 'images3\\BIR_MOD_

In [139]:
len(new_data['Normalized_Filename'].unique())

125

In [140]:
errors

[]

In [141]:
len(visited)

125

In [142]:
len(os.listdir("images2"))

125